<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>

# Google GenAI SDK - Building an Evaluator-Optimizer Agent

## Install Dependencies

In [ ]:
!pip install -q google-genai openinference-instrumentation-google-genai arize-phoenix-otel

## Connect to Arize Phoenix

In [ ]:
import os
from getpass import getpass

from phoenix.otel import register

if "PHOENIX_API_KEY" not in os.environ:
    os.environ["PHOENIX_API_KEY"] = getpass("Enter your Phoenix API key: ")

os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={os.environ['PHOENIX_API_KEY']}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com/"

tracer_provider = register(
    auto_instrument=True, project_name="google-genai-evaluator-optimizer-agent"
)
tracer = tracer_provider.get_tracer(__name__)

## Authenticate with Google Vertex AI

In [ ]:
from google import genai

In [ ]:
!gcloud auth login

In [ ]:
# Create a client using the Vertex AI API, you could also use the Google GenAI API instead here
client = genai.Client(vertexai=True, project="<ADD YOUR GCP PROJECT ID>", location="us-central1")

# Evaluator Optimizer

In [ ]:
# --- 1. Instantiate the model(s) ---
# The global `genai` configuration (or the `client` created in a previous
# notebook cell) is already set up, so we can request models right away.
story_model = "gemini-2.0-flash-001"
critic_model = "gemini-2.0-flash-001"

# --- Configuration for iterative improvement ---
max_iterations = 5  # Maximum number of critique/revision cycles
min_quality_threshold = 8  # Quality threshold on a scale of 1-10


# --- 2. Generate the initial short story ---
@tracer.agent()
def generate_story(story_model, client):
    initial_story = create_initial_story(story_model, client)
    return iteratively_improve_story(initial_story, story_model, critic_model, client)


@tracer.chain()
def create_initial_story(story_model, client):
    story_prompt = (
        "You are a creative short story writer.\n"
        "Write a brief, engaging story (3–4 paragraphs) about an unexpected "
        "adventure.\n"
        "Be imaginative but concise.\n"
        "User Input: "
    )
    story_response = client.models.generate_content(
        model=story_model,
        contents=story_prompt + input("Please enter a prompt or seed for your story: "),
    )
    current_story = story_response.text.strip()

    print("=== Initial Story ===\n")
    print(current_story)

    return current_story


@tracer.chain()
def iteratively_improve_story(current_story, story_model, critic_model, client):
    iteration = 0
    story_quality = 0  # Initial quality score

    while iteration < max_iterations and story_quality < min_quality_threshold:
        iteration += 1
        print(f"\n--- Iteration {iteration} ---")

        # Generate critique and extract quality score
        critique_text, story_quality = critique_story(
            current_story, critic_model, client, iteration
        )

        # Check if revision is needed
        if story_quality >= min_quality_threshold:
            print(
                f"\nStory quality ({story_quality}/10) meets or exceeds threshold ({min_quality_threshold}/10)."
            )
            print("No further revisions needed.")
            break

        # Improve the story based on the critique
        current_story = revise_story(current_story, critique_text, story_model, client, iteration)

    if iteration >= max_iterations and story_quality < min_quality_threshold:
        print(
            f"\n--- Maximum iterations ({max_iterations}) reached without meeting quality threshold ---"
        )

    print("\n--- Final Story ---")
    print(current_story)
    return current_story


@tracer.chain()
def critique_story(current_story, critic_model, client, iteration):
    print("\nGenerating critique and quality assessment...")
    critic_prompt = (
        "You are a literary critic.\n"
        "First, rate the story on a scale of 1-10 (where 10 is excellent).\n"
        "Then analyze the provided story for its strengths and weaknesses.\n"
        "Comment on plot, characters, and overall impact.\n"
        "If the story needs improvement, provide 2–3 specific suggestions.\n"
        "Format your response as follows:\n"
        "Quality Score: [1-10]\n"
        "Critique: [Your detailed critique]\n"
        "Suggestions: [Your specific suggestions if score is below 8]\n\n"
        f"Story:\n{current_story}"
    )
    critic_response = client.models.generate_content(model=critic_model, contents=critic_prompt)
    critique_text = critic_response.text.strip()

    # Display the critique
    print(f"\n=== Critique {iteration} ===\n")
    print(critique_text)

    # Extract quality score from critique
    try:
        # Try to extract the quality score from the critique
        quality_line = [line for line in critique_text.split("\n") if "Quality Score:" in line]
        if quality_line:
            story_quality = int(quality_line[0].split(":")[1].strip().split()[0])
        else:
            story_quality = 0  # Default if not found
    except (ValueError, IndexError):
        story_quality = 0  # Default if parsing fails

    print(f"\nQuality Score: {story_quality}/10")

    return critique_text, story_quality


@tracer.chain()
def revise_story(current_story, critique_text, story_model, client, iteration):
    print("\nGenerating revision...")
    revision_prompt = (
        "You are a creative short story writer.\n"
        "Revise the following story based *only* on the critique provided.\n"
        "Focus on addressing the specific suggestions mentioned in the critique.\n"
        "Do not introduce significant new elements not prompted by the critique.\n"
        "Make substantial improvements to raise the quality score.\n\n"
        f"Critique:\n{critique_text}\n\n"
        f"Original Story:\n{current_story}\n\n"
        "Please produce an improved version of the story, addressing the suggestions.\n"
        "Output *only* the revised story."
    )
    revision_response = client.models.generate_content(model=story_model, contents=revision_prompt)
    revised_story = revision_response.text.strip()

    # Display the improved story for this iteration
    print(f"\n=== Improved Story (Iteration {iteration}) ===\n")
    print(revised_story)

    return revised_story


generate_story(story_model, client)